In [1]:
import pandas as pd

# Load the CSV file
file_path = "/Users/tony/Desktop/outbound/final_data/final.csv"  # Update with the correct path if needed
data = pd.read_csv(file_path)

# Convert 'last_visited_date' column to datetime format (YYYY-MM-DD)
data['last_visited_date'] = pd.to_datetime(data['last_visited_date'], format='%Y-%m-%d', errors='coerce')

# Function to calculate days since last visit
def calculate_days_since(visit_date):
    today = pd.to_datetime('today').normalize()  # Get today's date without time
    return (today - visit_date).days if pd.notnull(visit_date) else None

# Apply function to update 'days_since_last_visit'
data['days_since_last_visit'] = data['last_visited_date'].apply(calculate_days_since)
data.fillna(0, inplace=True)


# Save the updated DataFrame back to CSV
data.to_csv(file_path, index=False)

print(f"Successfully updated {file_path} with recalculated 'days_since_last_visit' values.")

Successfully updated /Users/tony/Desktop/outbound/final_data/final.csv with recalculated 'days_since_last_visit' values.


In [2]:
import sqlite3
import pandas as pd

csv_path = "/Users/tony/Desktop/outbound/final_data/final.csv"
db_path = "/Users/tony/Desktop/outbound/final_data/lead_data.db"

def create_and_insert_lead_data(csv_path, db_path):
    # Load CSV
    data = pd.read_csv(csv_path)

    # Normalize column names to lowercase and remove duplicates
    data.columns = data.columns.str.lower()
    data = data.loc[:, ~data.columns.duplicated()]  # Drop duplicate columns

    # Convert last_visited_date to datetime if it exists
    if 'last_visited_date' in data.columns:
        data['last_visited_date'] = pd.to_datetime(data['last_visited_date'], errors='coerce')

    # Connect to SQLite
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Drop table if it exists
    cursor.execute("DROP TABLE IF EXISTS lead_data")

    # Define SQL table schema dynamically
    column_types = {
        'INTEGER': ['days_since_last_visit', 'total_score', 'employee_count', 'time_spent_on_site', 'visit_count', 
                    'founded_year', 'industry_score', 'location_score', 'role_score', 'number_of_visit_score',
                    'visit_duration_score', 'employee_count_score', 'contact_score', 'last_visit_score'],
        'REAL': ['revenue_in_million'],
        'TEXT': ['lead_id', 'domain', 'name', 'contact_email', 'mobile_number', 'position', 'linkedin_profile', 
                 'location', 'department', 'seniority', 'industry', 'contact_status', 'company_name', 'company_linkedin', 
                 'lead_url']
    }

    create_statement = "CREATE TABLE lead_data ("
    for col in data.columns:
        col_type = 'TEXT'  # Default type
        for dtype, cols in column_types.items():
            if col in cols:
                col_type = dtype
                break
        create_statement += f"{col} {col_type}, "
    
    create_statement = create_statement.rstrip(', ') + ")"
    cursor.execute(create_statement)

    # Insert data
    data.to_sql("lead_data", conn, if_exists="replace", index=False)

    # Commit and close
    conn.commit()
    conn.close()
    print("Data successfully inserted into SQLite database!")

# Run function
create_and_insert_lead_data(csv_path, db_path)


Data successfully inserted into SQLite database!
